In [10]:
from typing import Any, Optional
import os
import sys
import re
import ast

import numpy as np
import pandas as pd

from scipy.optimize import minimize

from definition import Los, Trip
from abc_mc import ModeChoiceModel

## Model Definition

TODO
- Implement additional mode choice models (e.g., Nested Logit, Mixed Logit)
- Add two methods for the model class (calculate_mode_probability, isValid)

In [11]:
# Example of Model Definition (MNL)
class MNL(ModeChoiceModel):
    def calculate_mode_probability(self, los: Los, params: np.ndarray) -> dict[int, float]:
        """
        Calculate mode choice probabilities using Multinomial Logit model.

        Args:
            los (Los): Level of Service data for the trip.
            params (np.ndarray): Model parameters corresponding to los.attribute_names.

        Returns:
            dict[int, float]: Dictionary of mode choice probabilities.
        """
        utilities = np.full(len(los.availability), -np.inf, dtype=np.float32)
        for i, mode in enumerate(los.availability.keys()):
            available = los.availability[int(mode)]
            attr_values = los.attributes[int(mode)]
            if available:
                utilities[i] = np.dot(params, attr_values)

        utilities = np.array(utilities)
        exp_utilities = np.exp(utilities - np.max(utilities))
        probabilities = exp_utilities / np.sum(exp_utilities)

        probabilities_dict = {mode: prob for mode, prob in zip(los.availability.keys(), probabilities)}

        return probabilities_dict
    

    def isValid(self, los: Los, params: np.ndarray) -> bool:
        return len(params) == len(los.attribute_names)

## Estimation

TODO
- Change path to los data and trip data
- Change model definition

In [13]:
LOS_FILE = "../data/input/test/los.csv"
TRIP_FILE = "../data/input/test/trip.csv"

# Create model
model = MNL()

# Read data
df_los = pd.read_csv(LOS_FILE)
df_trip = pd.read_csv(TRIP_FILE)

los_dict = {
    (row["OZone"], row["DZone"]): Los.from_dict(row)
    for row in df_los.to_dict(orient="records")
}

df_trip = df_trip.astype({
    "TripID": int,
    "DepartureTime": "datetime64[ns]",
    "ArrivalTime": "datetime64[ns]",
    "OZone": int,
    "DZone": int,
    "Mode": int
})

trips = [
    Trip(
        trip_id=row["TripID"],
        dep_time=row["DepartureTime"],
        arr_time=row["ArrivalTime"],
        o_zone=row["OZone"],
        d_zone=row["DZone"],
        model=model,  # dependency injection
        mode=row["Mode"]
    )
    for row in df_trip.to_records(index=False)
]

In [14]:
# Define functions for estimation
def compute_minus_ll(params: np.ndarray) -> float:
    ll = 0.0
    for trip in trips:
        los = trip.get_los(los_dict)
        if los is not None:
            ll += trip.calculate_log_likelihood(los, params)
    return -ll

def compute_hessian(params: np.ndarray) -> np.ndarray:
    h = 10 ** -4  # 数値微分用の微小量
    n = len(params)
    res = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            e_i, e_j = np.zeros(n), np.zeros(n)
            e_i[i] = 1
            e_j[j] = 1

            res[i][j] = (-compute_minus_ll(params + h * e_i + h * e_j)
                        + compute_minus_ll(params + h * e_i - h * e_j)
                        + compute_minus_ll(params - h * e_i + h * e_j)
                        - compute_minus_ll(params - h * e_i - h * e_j)) / (4 * h * h)
    return res

def tval(x: np.ndarray) -> np.ndarray:
    return x / np.sqrt(-np.diag(np.linalg.inv(compute_hessian(x))))

In [15]:
# Perform Estimation
x0 = np.zeros(len(los_dict[list(los_dict.keys())[0]].attribute_names))
res = minimize(compute_minus_ll, x0, method="Nelder-Mead")
t_val = tval(res.x)
LL0 = -compute_minus_ll(x0)
LL = -compute_minus_ll(res.x)
rho2 = 1 - LL / LL0
adj_rho2 = 1 - (LL - len(res.x)) / LL0
aic = -2 * LL + 2 * len(res.x)

result_str = f"""
sample number = {len(trips)}
    variables = {', '.join(los_dict[list(los_dict.keys())[0]].attribute_names)}
    parameter = {', '.join(map(str, res.x))}
        t value = {', '.join(map(str, t_val))}
            L0 = {LL0}
            LL = {LL}
            rho2 = {rho2}
adjusted rho2 = {adj_rho2}
            AIC = {aic}
"""
print(result_str)


sample number = 340
    variables = Time
    parameter = -0.030000000000000006
        t value = -8.886211562062822
            L0 = -479.0426747327091
            LL = -393.3092821360348
            rho2 = 0.1789681736486438
adjusted rho2 = 0.17688067695420429
            AIC = 788.6185642720696



## Simulation

TODO
- Change path to los data, trip data, parameter file, output dir
- Change model definition

In [16]:
LOS_FILE = "../data/input/test/los.csv"
TRIP_FILE = "../data/input/test/trip.csv"
PARAMETER_FILE = "../data/output/test/result.txt"
OUTPUT_DIR = "../data/output/test"

# Create model
model = MNL()

# Read data
df_los = pd.read_csv(LOS_FILE)
df_trip = pd.read_csv(TRIP_FILE)

los_dict = {
    (row["OZone"], row["DZone"]): Los.from_dict(row)
    for row in df_los.to_dict(orient="records")
}

df_trip = df_trip.astype({
    "TripID": int,
    "DepartureTime": "datetime64[ns]",
    "ArrivalTime": "datetime64[ns]",
    "OZone": int,
    "DZone": int
})

# Read parameters
with open(PARAMETER_FILE, encoding="utf-8") as f:
    text = f.read()

match = re.search(r"parameter\s*=\s*(\[[^\]]+\])", text)
if match:
    param_list = ast.literal_eval(match.group(1))  # list[float]
else:
    raise ValueError("Failed to extract parameters from input/result.txt")
params = np.array(param_list, dtype=np.float32)

In [18]:
# Perform Simulation
modes = []
for trip in trips:
    los = trip.get_los(los_dict)
    if los is not None:
        mode = trip.choose_mode(los, params)
        modes.append(mode)

df_trip["Mode"] = modes
df_trip.to_csv(os.path.join(OUTPUT_DIR, "trip_simulated.csv"), index=False)
print(df_trip.head())

   TripID       DepartureTime         ArrivalTime  OZone  DZone  Mode
0  255461 2009-10-31 06:19:00 2009-10-31 07:50:01    205    113   100
1  255291 2009-10-29 07:36:03 2009-10-29 08:21:20    115    104   410
2  256382 2009-11-10 06:55:33 2009-11-10 07:58:02    207    104   200
3  257459 2009-11-21 10:40:54 2009-11-21 12:31:44    112    206   200
4  257040 2009-11-17 12:09:25 2009-11-17 12:52:15    104    104   420
